In [21]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime

def format_discount_date(raw_text):
    try:
        dt = datetime.strptime(raw_text.strip(), "%Y/%m/%d %H:%M")
        return dt.strftime("%Y-%m-%d %H:%M")
    except Exception as e:
        print(f"[오류] 날짜 변환 실패: {raw_text} → {e}")
        return None

options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

browser = uc.Chrome(options=options)
all_dfs = []

# 🔁 원하는 페이지 범위 지정
for page in range(12,21):
    url = f"https://store.nintendo.co.kr/all-product?p={page}&product_list_limit=24"
    browser.get(url)
    print(f"\n📄 {page} 페이지 접근 중...")

    time.sleep(random.uniform(5, 8))

    if page == 1:
        try:
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "popup-close"))
            )
            browser.execute_script("document.querySelector('.popup-close').click();")
            print("✅ 팝업 닫음")
        except TimeoutException:
            print("✅ 팝업 없음")

    soup = BeautifulSoup(browser.page_source, "html.parser")
    product_list = soup.select("div > ol > li")
    print(f"🛍️ 상품 수: {len(product_list)}")

    products = []

    for product in product_list:
        try:
            name_tag = product.select_one('strong > a.product-item-link')
            game_name = name_tag.text.strip()
            game_url = name_tag.get('href')

            img_tag = product.select_one('span.product-image-container img')
            game_img_url = img_tag.get('src') if img_tag else None

            release_date = None
            release_tag = product.select_one('div.category-product-item-released')
            if release_tag:
                release_text = release_tag.text.strip().replace('발매', '').strip()
                try:
                    date_obj = datetime.strptime(release_text, "%y.%m.%d")
                    release_date = date_obj.strftime("%Y-%m-%d")
                except ValueError:
                    pass

            price_tags = product.select('span.price')
            if product.select('span.old-price') and price_tags:
                original_price = product.select_one('span.old-price span.price-wrapper span.price').text.strip()
                discount_price = price_tags[0].text.strip()
            elif price_tags:
                original_price = discount_price = price_tags[0].text.strip()
            else:
                original_price = discount_price = None

            products.append({
                'game_name': game_name,
                'game_url': game_url,
                'game_img_url': game_img_url,
                'release_date': release_date,
                'original_price': original_price,
                'discount_price': discount_price
            })
        except Exception as e:
            print(f"[상품 파싱 오류] {e}")
            continue

    df = pd.DataFrame(products)

    # 상세 정보 수집
    for idx, row in df.iterrows():
        browser.get(row['game_url'])
        time.sleep(random.uniform(3, 4))

        discount_startdate = discount_enddate = genre = maker = play_number = game_language = None
        product_type = 'SWC'

        soup = BeautifulSoup(browser.page_source, 'html.parser')

        try:
            discount_start_tag = soup.select_one('span.special-period-start')
            discount_end_tag = soup.select_one('span.special-period-end')
            discount_startdate = format_discount_date(discount_start_tag.text) if discount_start_tag else None
            discount_enddate = format_discount_date(discount_end_tag.text) if discount_end_tag else None

            genre_tag = soup.select_one('div.product-attribute.game_category > div.product-attribute-val > div.attribute-item-val')
            genre = genre_tag.text.strip() if genre_tag else None

            maker_tag = soup.select_one('div.product-attribute.publisher > div.product-attribute-val > div.attribute-item-val')
            maker = maker_tag.text.strip() if maker_tag else None

            player_tag = soup.select_one('div.product-attribute.no_of_players > div.product-attribute-val')
            if player_tag:
                text = player_tag.text.strip()
                play_number = text.split('✕')[1].strip() if '✕' in text else None

            if '실물 패키지 상품' in soup.text:
                product_type = 'PHC'

            lang_tag = soup.select_one('div.product-attribute.supported_languages > div.product-attribute-val > div.attribute-item-val')
            game_language = lang_tag.text.strip() if lang_tag else None

        except Exception as e:
            print(f"[상세 파싱 오류] {e}")

        df.at[idx, 'discount_startdate'] = discount_startdate
        df.at[idx, 'discount_enddate'] = discount_enddate
        df.at[idx, 'genre'] = genre
        df.at[idx, 'maker'] = maker
        df.at[idx, 'play_number'] = play_number
        df.at[idx, 'product_type'] = product_type
        df.at[idx, 'game_language'] = game_language

        print(f"✅ {idx}: {row['game_name']} 수집 완료 | 장르: {genre} | 언어: {game_language}")

    all_dfs.append(df)

browser.quit()

# 최종 저장
final_df = pd.concat(all_dfs, ignore_index=True)




📄 12 페이지 접근 중...
🛍️ 상품 수: 24
✅ 0: Book Supermarket: Library Manager Simulator - 도서 슈퍼마켓: 도서관 관리자 시뮬레이터 수집 완료 | 장르: None | 언어: None
✅ 1: Motorbikes Pro 2025 수집 완료 | 장르: None | 언어: None
✅ 2: FATE: Reawakened 수집 완료 | 장르: 액션, RPG, 어드벤처 | 언어: 한국어, 영어, 스페인어, 프랑스어, 독일어, 일본어, 중국어
✅ 3: Mine Clicker - Base Craft Building Game 수집 완료 | 장르: 어드벤처, 전략, 시뮬레이션, 보드 | 언어: 영어
✅ 4: The Exorcist: Gravebound Horror 수집 완료 | 장르: 어드벤처, 전략, 퍼즐, 시뮬레이션 | 언어: 영어, 스페인어, 러시아어, 일본어, 중국어
✅ 5: ESCAPE SITE 13 수집 완료 | 장르: 액션, RPG, 어드벤처, 시뮬레이션 | 언어: 영어, 일본어, 중국어
✅ 6: 한국 드론 플라잉 투어 강화도 수집 완료 | 장르: 아케이드, 시뮬레이션, 기타 | 언어: 한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 네덜란드어, 포르투갈어, 러시아어, 일본어, 중국어
✅ 7: Wine Factory Simulator 수집 완료 | 장르: 파티, 시뮬레이션, 트레이닝, 학습 | 언어: 영어
✅ 8: 귀여운 틀린 그림 찾기 수집 완료 | 장르: 파티, 트레이닝, 학습, 실용 | 언어: 한국어, 영어, 일본어, 중국어
✅ 9: Sugoi Girls: Sassy Spy 수집 완료 | 장르: 액션, 퍼즐, 기타, 실용 | 언어: 한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 일본어, 중국어
✅ 10: Fallen Legion Revenants 수집 완료 | 장르: 액션, RPG, 어드벤처 | 언어: 영어, 중국어
✅ 11: BEAST: 바이오 엑소 아레나 슈트 팀 (Bio Exo

In [22]:
final_df

,game_name,game_url,game_img_url,release_date,original_price,discount_price,discount_startdate,discount_enddate,genre,maker,play_number,product_type,game_language
0,Book Supermarket: Library Manager Simulator - ...,https://store.nintendo.co.kr/70010000094328,https://store.nintendo.co.kr/media/catalog/pro...,2025-03-13,"₩7,990","₩7,990",None,None,None,None,None,SWC,None
1,Motorbikes Pro 2025,https://store.nintendo.co.kr/70010000094982,https://store.nintendo.co.kr/media/catalog/pro...,2025-03-13,"₩18,600","₩18,600",None,None,None,None,None,SWC,None
2,FATE: Reawakened,https://store.nintendo.co.kr/70010000083168,https://store.nintendo.co.kr/media/catalog/pro...,2025-03-13,"₩31,400","₩31,400",None,None,"액션, RPG, 어드벤처",gamigo US,1 ~ 1,SWC,"한국어, 영어, 스페인어, 프랑스어, 독일어, 일본어, 중국어"
3,Mine Clicker - Base Craft Building Game,https://store.nintendo.co.kr/70010000092603,https://store.nintendo.co.kr/media/catalog/pro...,2025-03-13,"₩9,990","₩9,990",None,None,"어드벤처, 전략, 시뮬레이션, 보드",Grizzly Games,1 ~ 1,SWC,영어
4,The Exorcist: Gravebound Horror,https://store.nintendo.co.kr/70010000094289,https://store.nintendo.co.kr/media/catalog/pro...,2025-03-13,"₩20,000","₩6,000",2025-04-10 00:00,2025-04-24 00:59,"어드벤처, 전략, 퍼즐, 시뮬레이션",Evgheni Carasiov,1 ~ 1,SWC,"영어, 스페인어, 러시아어, 일본어, 중국어"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Connect the Circuit 1000,https://store.nintendo.co.kr/70010000089750,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩4,500","₩4,500",None,None,"아케이드, 퍼즐, 시뮬레이션, 보드",SUCCESS,1 ~ 1,SWC,"한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 일본어, 중국어"
212,Sokomonster,https://store.nintendo.co.kr/70010000090333,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩4,070","₩4,070",None,None,"아케이드, 퍼즐",Afil Games,1 ~ 1,SWC,"영어, 포르투갈어"
213,Bubble Shooter World,https://store.nintendo.co.kr/70010000090560,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩4,300","₩4,300",None,None,"액션, 아케이드, 퍼즐, 슈팅",Silesia Games,1 ~ 1,SWC,"영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 러시아어"
214,Cubic Run,https://store.nintendo.co.kr/70010000089290,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩2,100","₩2,100",None,None,"액션, 아케이드, 기타, 트레이닝",Webnetic,1 ~ 1,SWC,영어


In [23]:
final_df.to_csv('nintendo_games2.csv', mode='a', header=False, index=False, encoding ='utf-8-sig')
print("✅ 저장 완료: nintendo_games2.csv")
pd.read_csv('nintendo_games2.csv')

✅ 저장 완료: nintendo_games2.csv


,game_name,game_url,game_img_url,release_date,original_price,discount_price,discount_startdate,discount_enddate,genre,maker,play_number,product_type,game_language
0,스노우 브라더스 2 스페셜 (SNOW BROS. 2 SPECIAL),https://store.nintendo.co.kr/70010000084494,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩44,800","₩44,800",NaN,NaN,"액션, 아케이드",Gravity,1 ~ 4,SWC,"한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 러시아어, ..."
1,퀴즈! 세계의 고양이 속담,https://store.nintendo.co.kr/70010000095482,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩3,900","₩1,950",2025-04-10 00:00,2025-05-04 23:59,"파티, 트레이닝, 학습, 실용",MASK,1 ~ 1,SWC,"한국어, 영어, 일본어, 중국어"
2,Pilo and the Holobook,https://store.nintendo.co.kr/70010000092824,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩18,850","₩5,990",2025-04-10 00:00,2025-05-06 23:59,"어드벤처, 아케이드, 퍼즐, 시뮬레이션",RedDeer.Games,1 ~ 1,SWC,"한국어, 영어, 프랑스어, 독일어, 일본어, 중국어"
3,Arcade Archives GANGBUSTERS,https://store.nintendo.co.kr/70010000090953,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩10,600","₩10,600",NaN,NaN,"액션, 아케이드, 슈팅",HAMSTER,1 ~ 2,SWC,"영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 일본어"
4,스이카 게임 (Suika Water World),https://store.nintendo.co.kr/70010000095523,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩4,490","₩4,490",NaN,NaN,"파티, 퍼즐, 학습",YoboGames,1 ~ 1,SWC,영어
...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,Connect the Circuit 1000,https://store.nintendo.co.kr/70010000089750,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩4,500","₩4,500",NaN,NaN,"아케이드, 퍼즐, 시뮬레이션, 보드",SUCCESS,1 ~ 1,SWC,"한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 일본어, 중국어"
404,Sokomonster,https://store.nintendo.co.kr/70010000090333,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩4,070","₩4,070",NaN,NaN,"아케이드, 퍼즐",Afil Games,1 ~ 1,SWC,"영어, 포르투갈어"
405,Bubble Shooter World,https://store.nintendo.co.kr/70010000090560,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩4,300","₩4,300",NaN,NaN,"액션, 아케이드, 퍼즐, 슈팅",Silesia Games,1 ~ 1,SWC,"영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 러시아어"
406,Cubic Run,https://store.nintendo.co.kr/70010000089290,https://store.nintendo.co.kr/media/catalog/pro...,2025-01-30,"₩2,100","₩2,100",NaN,NaN,"액션, 아케이드, 기타, 트레이닝",Webnetic,1 ~ 1,SWC,영어


In [ ]:
#nintendo_games:  1-5
#nintendo_games2
#nintendo_gmaes_11-20


In [50]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime
from fake_useragent import UserAgent

def format_discount_date(raw_text):
    try:
        dt = datetime.strptime(raw_text.strip(), "%Y/%m/%d %H:%M")
        return dt.strftime("%Y-%m-%d %H:%M")
    except Exception as e:
        print(f"[오류] 날짜 변환 실패: {raw_text} → {e}")
        return None

options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
ua = UserAgent()
options.add_argument(f"user-agent={ua.random}")

browser = uc.Chrome(options=options)
all_dfs = []

# 🔁 원하는 페이지 범위 지정
for page in range(101, 150): #101-149
    url = f"https://store.nintendo.co.kr/all-product?p={page}&product_list_limit=24"
    browser.get(url)
    print(f"\n📄 {page} 페이지 접근 중...")
    time.sleep(random.uniform(7, 10))  # 페이지 이동 시 딜레이 증가

    if page == 1:
        try:
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "popup-close"))
            )
            browser.execute_script("document.querySelector('.popup-close').click();")
            print("✅ 팝업 닫음")
        except TimeoutException:
            print("✅ 팝업 없음")

    soup = BeautifulSoup(browser.page_source, "html.parser")
    product_list = soup.select("div > ol > li")
    print(f"🛍️ 상품 수: {len(product_list)}")

    products = []

    for product in product_list:
        try:
            name_tag = product.select_one('strong > a.product-item-link')
            game_name = name_tag.text.strip()
            game_url = name_tag.get('href')

            img_tag = product.select_one('span.product-image-container img')
            game_img_url = img_tag.get('src') if img_tag else None

            release_date = None
            release_tag = product.select_one('div.category-product-item-released')
            if release_tag:
                release_text = release_tag.text.strip().replace('발매', '').strip()
                try:
                    date_obj = datetime.strptime(release_text, "%y.%m.%d")
                    release_date = date_obj.strftime("%Y-%m-%d")
                except ValueError:
                    pass

            price_tags = product.select('span.price')
            if product.select('span.old-price') and price_tags:
                original_price = product.select_one('span.old-price span.price-wrapper span.price').text.strip()
                discount_price = price_tags[0].text.strip()
            elif price_tags:
                original_price = discount_price = price_tags[0].text.strip()
            else:
                original_price = discount_price = None

            products.append({
                'game_name': game_name,
                'game_url': game_url,
                'game_img_url': game_img_url,
                'release_date': release_date,
                'original_price': original_price,
                'discount_price': discount_price
            })
        except Exception as e:
            print(f"[상품 파싱 오류] {e}")
            continue

    df = pd.DataFrame(products)

    # 상세 정보 수집
    for idx, row in df.iterrows():
        time.sleep(random.uniform(4, 6))  # 상세 페이지 접근 시 딜레이 증가
        try:
            browser.get(row['game_url'])
        except Exception as e:
            print(f"[상세 페이지 접근 오류]: {e}")
            continue

        discount_startdate = discount_enddate = genre = maker = play_number = game_language = None
        product_type = 'SWC'

        soup = BeautifulSoup(browser.page_source, 'html.parser')

        try:
            discount_start_tag = soup.select_one('span.special-period-start')
            discount_end_tag = soup.select_one('span.special-period-end')
            discount_startdate = format_discount_date(discount_start_tag.text) if discount_start_tag else None
            discount_enddate = format_discount_date(discount_end_tag.text) if discount_end_tag else None

            genre_tag = soup.select_one('div.product-attribute.game_category > div.product-attribute-val > div.attribute-item-val')
            genre = genre_tag.text.strip() if genre_tag else None

            maker_tag = soup.select_one('div.product-attribute.publisher > div.product-attribute-val > div.attribute-item-val')
            maker = maker_tag.text.strip() if maker_tag else None

            player_tag = soup.select_one('div.product-attribute.no_of_players > div.product-attribute-val')
            if player_tag:
                text = player_tag.text.strip()
                play_number = text.split('✕')[1].strip() if '✕' in text else None

            if '실물 패키지 상품' in soup.text:
                product_type = 'PHC'

            lang_tag = soup.select_one('div.product-attribute.supported_languages > div.product-attribute-val > div.attribute-item-val')
            game_language = lang_tag.text.strip() if lang_tag else None

        except Exception as e:
            print(f"[상세 파싱 오류] {e}")

        df.at[idx, 'discount_startdate'] = discount_startdate
        df.at[idx, 'discount_enddate'] = discount_enddate
        df.at[idx, 'genre'] = genre
        df.at[idx, 'maker'] = maker
        df.at[idx, 'play_number'] = play_number
        df.at[idx, 'product_type'] = product_type
        df.at[idx, 'game_language'] = game_language

        print(f"✅ {idx}: {row['game_name']} 수집 완료 | 장르: {genre} | 언어: {game_language}")

    all_dfs.append(df)

browser.quit()

# 최종 저장
final_df = pd.concat(all_dfs, ignore_index=True)
print(final_df)


📄 101 페이지 접근 중...
🛍️ 상품 수: 24
✅ 0: Celeste 수집 완료 | 장르: 액션, 어드벤처, 기타 | 언어: 한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 러시아어, 일본어, 중국어
✅ 1: Raccoo Venture 수집 완료 | 장르: 액션, 어드벤처, 퍼즐, 기타 | 언어: 영어, 스페인어, 독일어, 포르투갈어
✅ 2: Bulldozer Tycoon: Construction Simulator 수집 완료 | 장르: 시뮬레이션 | 언어: 영어
✅ 3: MECHBLAZE 수집 완료 | 장르: 액션, 아케이드, 슈팅, 기타 | 언어: 영어, 독일어, 일본어, 중국어
✅ 4: 스미코구라시 다 함께 리듬 파티 수집 완료 | 장르: 파티, 음악, 기타 | 언어: 한국어
✅ 5: Airport 수집 완료 | 장르: 파티, 아케이드, 퍼즐, 시뮬레이션 | 언어: 영어, 스페인어, 프랑스어, 일본어, 중국어
✅ 6: Hentai Golf 수집 완료 | 장르: 스포츠, 아케이드, 시뮬레이션 | 언어: 한국어, 영어, 일본어, 중국어
✅ 7: 색칠하며 놀아요 동물원 수집 완료 | 장르: 기타, 트레이닝, 학습 | 언어: 한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 일본어, 중국어
✅ 8: City Bus Driving Simulator 수집 완료 | 장르: 액션, 시뮬레이션 | 언어: 영어
✅ 9: Potion Craft: Alchemist Simulator 수집 완료 | 장르: 시뮬레이션, 기타 | 언어: 한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 러시아어, 일본어, 중국어
✅ 10: Outer Wilds 수집 완료 | 장르: 어드벤처 | 언어: 한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 러시아어, 일본어, 중국어
✅ 11: Shanghai LEGEND 수집 완료 | 장르: 아케이드, 퍼즐, 보드, 트레이닝 | 언어: 영어, 일본어, 중국어
✅ 12: 트

In [51]:
final_df.to_csv('nintendo_games2.csv', mode='a', header=False, index=False, encoding ='utf-8-sig')
print("✅ 저장 완료: nintendo_games2.csv")
pd.read_csv('nintendo_games2.csv')

✅ 저장 완료: nintendo_games2.csv


,game_name,game_url,game_img_url,release_date,original_price,discount_price,discount_startdate,discount_enddate,genre,maker,play_number,product_type,game_language
0,스노우 브라더스 2 스페셜 (SNOW BROS. 2 SPECIAL),https://store.nintendo.co.kr/70010000084494,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩44,800","₩44,800",NaN,NaN,"액션, 아케이드",Gravity,1 ~ 4,SWC,"한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 러시아어, ..."
1,퀴즈! 세계의 고양이 속담,https://store.nintendo.co.kr/70010000095482,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩3,900","₩1,950",2025-04-10 00:00,2025-05-04 23:59,"파티, 트레이닝, 학습, 실용",MASK,1 ~ 1,SWC,"한국어, 영어, 일본어, 중국어"
2,Pilo and the Holobook,https://store.nintendo.co.kr/70010000092824,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩18,850","₩5,990",2025-04-10 00:00,2025-05-06 23:59,"어드벤처, 아케이드, 퍼즐, 시뮬레이션",RedDeer.Games,1 ~ 1,SWC,"한국어, 영어, 프랑스어, 독일어, 일본어, 중국어"
3,Arcade Archives GANGBUSTERS,https://store.nintendo.co.kr/70010000090953,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩10,600","₩10,600",NaN,NaN,"액션, 아케이드, 슈팅",HAMSTER,1 ~ 2,SWC,"영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 일본어"
4,스이카 게임 (Suika Water World),https://store.nintendo.co.kr/70010000095523,https://store.nintendo.co.kr/media/catalog/pro...,2025-04-10,"₩4,490","₩4,490",NaN,NaN,"파티, 퍼즐, 학습",YoboGames,1 ~ 1,SWC,영어
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,Piczle Lines 2:퍼즐버스 속으로,https://store.nintendo.co.kr/70010000057974,https://store.nintendo.co.kr/media/catalog/pro...,2023-02-22,"₩14,000","₩3,500",2025-04-17 00:00,2025-05-07 23:59,"액션, 퍼즐, 시뮬레이션",RAINYFROG,1 ~ 1,SWC,"한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 일본어, 중국어"
3516,네온의 무법자 (Dust & Neon),https://store.nintendo.co.kr/70010000052940,https://store.nintendo.co.kr/media/catalog/pro...,2023-02-16,"₩21,500","₩4,300",2025-04-19 00:00,2025-05-09 23:59,"액션, RPG, 어드벤처, 슈팅",Rogue Games,1 ~ 1,SWC,"한국어, 영어, 스페인어, 프랑스어, 독일어, 이탈리아어, 포르투갈어, 일본어, 중국어"
3517,모자이크 크로니클 디럭스,https://store.nintendo.co.kr/70010000061473,https://store.nintendo.co.kr/media/catalog/pro...,2023-02-16,"₩16,060","₩1,990",2025-04-23 00:00,2025-05-19 23:59,"어드벤처, 퍼즐",RedDeer.Games,1 ~ 1,SWC,"영어, 러시아어, 일본어"
3518,nOS 뉴 오퍼레이팅 시스템,https://store.nintendo.co.kr/70010000058458,https://store.nintendo.co.kr/media/catalog/pro...,2023-02-16,"₩51,980","₩51,980",NaN,NaN,"시뮬레이션, 기타, 학습, 실용",RedDeer.Games,1 ~ 1,SWC,"한국어, 영어, 독일어, 일본어, 중국어"


In [52]:
import pandas as pd

# 병합할 파일 리스트
file_list = [
    "nintendo_games.csv",
    "nintendo_games2.csv",
    "nintendo_games151_160.csv",
    "nintendo_games161_170.csv",
    "nintendo_games171_180.csv",
    "nintendo_games181_190.csv",
    "nintendo_games191_200.csv",
    "nintendo_games201_210.csv",
    "nintendo_games211_220.csv",
    "nintendo_games221_230.csv",
    "nintendo_games231_240.csv",
    "nintendo_games241_258.csv"
]

# 파일들을 순서대로 읽고 하나의 데이터프레임으로 합치기
df_list = [pd.read_csv(file) for file in file_list]
merged_df = pd.concat(df_list, ignore_index=True)

# 결과 확인
print(merged_df.head())
print(f"총 행 개수: {len(merged_df)}")


                     game_name                                     game_url  \
0              GRADIUS ORIGINS  https://store.nintendo.co.kr/70010000074360   
1        The Wandering Village  https://store.nintendo.co.kr/70010000082518   
2     HUNTER×HUNTER NEN×IMPACT  https://store.nintendo.co.kr/70010000084029   
3  Shadow Labyrinth (쉐도우 라비린스)  https://store.nintendo.co.kr/70010000077048   
4                    다마고치 원더 샵  https://store.nintendo.co.kr/70010000096183   

                                        game_img_url release_date  \
0  https://store.nintendo.co.kr/media/catalog/pro...   2025-08-07   
1  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
2  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
3  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
4  https://store.nintendo.co.kr/media/catalog/pro...   2025-06-26   

  original_price discount_price discount_startdate  discount_enddate  \
0        ₩53,800        ₩48,420   2025

In [54]:
# 최신순으로 정렬
sorted_df = merged_df.sort_values(by='release_date', ascending=False)

# 결과 확인
print(sorted_df.head())


                     game_name                                     game_url  \
0              GRADIUS ORIGINS  https://store.nintendo.co.kr/70010000074360   
1        The Wandering Village  https://store.nintendo.co.kr/70010000082518   
2     HUNTER×HUNTER NEN×IMPACT  https://store.nintendo.co.kr/70010000084029   
3  Shadow Labyrinth (쉐도우 라비린스)  https://store.nintendo.co.kr/70010000077048   
4                    다마고치 원더 샵  https://store.nintendo.co.kr/70010000096183   

                                        game_img_url release_date  \
0  https://store.nintendo.co.kr/media/catalog/pro...   2025-08-07   
1  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
2  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
3  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
4  https://store.nintendo.co.kr/media/catalog/pro...   2025-06-26   

  original_price discount_price discount_startdate  discount_enddate  \
0        ₩53,800        ₩48,420   2025

In [55]:
sorted_df.to_csv("df_1.csv", index = False)
df = pd.read_csv("df_1.csv")

In [56]:
# game_name과 release_date가 모두 동일한 행 제거
deduped_df = df.drop_duplicates(subset=['game_name', 'release_date'])

# 결과 확인
print(deduped_df.head())
print(f"중복 제거 후 행 개수: {len(deduped_df)}")


                     game_name                                     game_url  \
0              GRADIUS ORIGINS  https://store.nintendo.co.kr/70010000074360   
1        The Wandering Village  https://store.nintendo.co.kr/70010000082518   
2     HUNTER×HUNTER NEN×IMPACT  https://store.nintendo.co.kr/70010000084029   
3  Shadow Labyrinth (쉐도우 라비린스)  https://store.nintendo.co.kr/70010000077048   
4                    다마고치 원더 샵  https://store.nintendo.co.kr/70010000096183   

                                        game_img_url release_date  \
0  https://store.nintendo.co.kr/media/catalog/pro...   2025-08-07   
1  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
2  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
3  https://store.nintendo.co.kr/media/catalog/pro...   2025-07-17   
4  https://store.nintendo.co.kr/media/catalog/pro...   2025-06-26   

  original_price discount_price discount_startdate  discount_enddate  \
0        ₩53,800        ₩48,420   2025

In [63]:
# original_price가 NaN이 아닌 행만 남기기 (즉, NaN인 행 제거)
filtered_df = deduped_df[deduped_df['original_price'].notna()]
print(f"필터링 후 행 개수: {len(filtered_df)}")


필터링 후 행 개수: 6069


In [66]:
# 원화 기호(₩) 제거 후 숫자(float)로 변환
filtered_df.loc[:, 'discount_price'] = (
    filtered_df['discount_price'].astype(str).str.replace('₩', '').str.replace(',', '').astype(float)
)
filtered_df.loc[:, 'original_price'] = (
    filtered_df['original_price'].astype(str).str.replace('₩', '').str.replace(',', '').astype(float)
)

# 결과 확인
print(filtered_df[['discount_price', 'original_price']].head())


  discount_price original_price
0        48420.0        53800.0
1        28800.0        32000.0
2        69800.0        69800.0
3        39800.0        39800.0
4        49800.0        49800.0


In [78]:
# discount_price를 float로 변환 (변환 실패 시 NaN으로 처리)
filtered_df.loc[:, 'discount_price'] = pd.to_numeric(filtered_df['discount_price'], errors='coerce')

# game_name에 "OLED 모델", "에디션", "세트", "선불", "패키지"가 포함된 행 제거
top_filtered = top_filtered[~top_filtered['game_name'].str.contains('OLED 모델|에디션|세트|선불|패키지', na=False)]

# discount_price가 100,000 이상인 행 제거
top_filtered = top_filtered[top_filtered['discount_price'] < 100000]

# 결과 확인
print(len(top_filtered))


5998


In [79]:
# release_date 기준으로 최신순으로 정렬
top_filtered_sorted = top_filtered.sort_values(by='release_date', ascending=False)

# 결과를 df_2.csv로 저장
top_filtered_sorted.to_csv('df_2.csv', index=False)

In [ ]:
#공짜 게임 지우기